In [1]:
#Preprocessing
#import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
chess_data = pd.read_csv('./datacleaned.csv')

In [3]:
chess_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19113 entries, 0 to 19112
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      19113 non-null  int64 
 1   black_rating    19113 non-null  int64 
 2   id              19113 non-null  object
 3   rated           19113 non-null  bool  
 4   turns           19113 non-null  int64 
 5   victory_status  19113 non-null  object
 6   white_rating    19113 non-null  int64 
 7   winner          19113 non-null  object
dtypes: bool(1), int64(4), object(3)
memory usage: 1.0+ MB


In [5]:
chess_data = chess_data.drop(chess_data.columns[[0,2]], axis=1)

In [6]:
chess_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19113 entries, 0 to 19112
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   black_rating    19113 non-null  int64 
 1   rated           19113 non-null  bool  
 2   turns           19113 non-null  int64 
 3   victory_status  19113 non-null  object
 4   white_rating    19113 non-null  int64 
 5   winner          19113 non-null  object
dtypes: bool(1), int64(3), object(2)
memory usage: 765.4+ KB


In [7]:
dummy = pd.get_dummies(chess_data['victory_status'])

In [8]:
print(dummy)

       draw  mate  outoftime  resign
0         0     0          1       0
1         0     0          0       1
2         0     1          0       0
3         0     1          0       0
4         0     1          0       0
...     ...   ...        ...     ...
19108     0     0          0       1
19109     0     1          0       0
19110     0     1          0       0
19111     0     0          0       1
19112     0     1          0       0

[19113 rows x 4 columns]


In [9]:
chess_with_dummy = pd.concat([chess_data, dummy], axis=1)
chess_with_dummy.head()

,black_rating,rated,turns,victory_status,white_rating,winner,draw,mate,outoftime,resign
0,1191,False,13,outoftime,1500,white,0,0,1,0
1,1261,True,16,resign,1322,black,0,0,0,1
2,1500,True,61,mate,1496,white,0,1,0,0
3,1454,True,61,mate,1439,white,0,1,0,0
4,1469,True,95,mate,1523,white,0,1,0,0


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [14]:
scaler= StandardScaler()
scaler.fit(chess_with_dummy[['black_rating']])
print(scaler.mean_)
chess_with_dummy['black_ss'] = scaler.transform(chess_with_dummy[['black_rating']])


[1590.04551876]


In [15]:
scaler.fit(chess_with_dummy[['white_rating']])
print(scaler.mean_)
chess_with_dummy['white_ss'] = scaler.transform(chess_with_dummy[['white_rating']])
scaler.fit(chess_with_dummy[['turns']])
print(scaler.mean_)
chess_with_dummy['turns_ss'] = scaler.transform(chess_with_dummy[['turns']])

[1597.30000523]
[60.51383875]


In [16]:
print(chess_with_dummy[['turns', 'turns_ss', 'white_rating', 'white_ss', 'black_rating', 'black_ss']].head())

   turns  turns_ss  white_rating  white_ss  black_rating  black_ss
0     13 -1.418858          1500 -0.335500          1191 -1.373962
1     16 -1.329272          1322 -0.949261          1261 -1.132944
2     61  0.014518          1496 -0.349292          1500 -0.310038
3     61  0.014518          1439 -0.545834          1454 -0.468421
4     95  1.029825          1523 -0.256194          1469 -0.416774


In [17]:
y = chess_with_dummy.pop('winner')
X = chess_with_dummy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)